In [1]:
# ##### INSTALLATION
# import sys
# sys.path.append('/kaggle/input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master')

In [2]:
# from efficientnet_pytorch import EfficientNet

In [3]:
import pandas as pd, numpy as np, os
import matplotlib.pyplot as plt, gc
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import freqz

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import KFold, GroupKFold
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
CUDA_LAUNCH_BLOCKING = "1"

In [5]:
df = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
TARGETS = df.columns[-6:]
print('Train shape:', df.shape )
print('Targets', list(TARGETS))
df.head()

Train shape: (106800, 15)
Targets ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']


,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0


In [6]:
train = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_id':'first','spectrogram_label_offset_seconds':'min'})
train.columns = ['spec_id','min']

tmp = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_label_offset_seconds':'max'})
train['max'] = tmp

tmp = df.groupby('eeg_id')[['patient_id']].agg('first')
train['patient_id'] = tmp

tmp = df.groupby('eeg_id')[TARGETS].agg('sum')
for t in TARGETS:
    train[t] = tmp[t].values
    
y_data = train[TARGETS].values
y_data = y_data / y_data.sum(axis=1,keepdims=True)
train[TARGETS] = y_data

tmp = df.groupby('eeg_id')[['expert_consensus']].agg(pd.Series.mode)
train['target'] = tmp

train = train.reset_index()
print('Train non-overlapp eeg_id shape:', train.shape )
train.head()

Train non-overlapp eeg_id shape: (17089, 12)


,eeg_id,spec_id,min,max,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,target
0,568657,789577333,0.0,16.0,20654,0.0,0.000000,0.25,0.000000,0.166667,0.583333,Other
1,582999,1552638400,0.0,38.0,20230,0.0,0.857143,0.00,0.071429,0.000000,0.071429,LPD
2,642382,14960202,1008.0,1032.0,5955,0.0,0.000000,0.00,0.000000,0.000000,1.000000,Other
3,751790,618728447,908.0,908.0,38549,0.0,0.000000,1.00,0.000000,0.000000,0.000000,GPD
4,778705,52296320,0.0,0.0,40955,0.0,0.000000,0.00,0.000000,0.000000,1.000000,Other


In [7]:
IS_TRAINING=True

In [8]:
if IS_TRAINING:
    spectrograms = np.load('/kaggle/input/brain-spectrograms/specs.npy',allow_pickle=True).item()

## Dataloader

In [9]:
class CustomDataset(Dataset):
    def __init__(self, dataframe,specs,mode='Train', transform=None):
        self.dataframe = dataframe
        self.specs = specs
        self.mode = mode
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        
        row = self.dataframe.iloc[idx]
        if self.mode=='Test': 
                r = 0
        else: 
            r = int( (row['min'] + row['max'])//4 )
        X = np.zeros((100,300,4))#,dtype='float64')
        for k in range(4):
                # EXTRACT 300 ROWS OF SPECTROGRAM
                img = self.specs[row.spec_id][r:r+300,k*100:(k+1)*100].T
                
                # LOG TRANSFORM SPECTROGRAM
                img = np.clip(img,np.exp(-4),np.exp(8))
                img = np.log(img)
                
                # STANDARDIZE PER IMAGE
                ep = 1e-6
                m = np.nanmean(img.flatten())
                s = np.nanstd(img.flatten())
                img = (img-m)/(s+ep)
                img = np.nan_to_num(img, nan=0.0)
                
                # CROP TO 256 TIME STEPS
                X[:,:,k] = img#[:,22:-22] / 2.0
                
        X = torch.tensor(X,dtype=torch.float32)
        X = X.permute(2, 0, 1)
#         print('test',X.shape)
        if self.mode=='Test': 
            return X
        else:
            labels = row[TARGETS].values.astype(np.float32) #np.array(row[-6:]).reshape(6,1)
            labels = labels/np.sum(labels)
            return X, torch.tensor(labels,dtype=torch.float32)

# Scratch Efficient Implementation

In [10]:
basic_mb_params = [
    # k, channels(c), repeats(t), stride(s), kernel_size(k)
    [1, 16, 1, 1, 3],
    [6, 24, 2, 2, 3],
    [6, 40, 2, 2, 5],
    [6, 80, 3, 2, 3],
    [6, 112, 3, 1, 5],
    [6, 192, 4, 2, 5],
    [6, 320, 1, 1, 3],
]

alpha, beta = 1.2, 1.1

scale_values = {
    # (phi, resolution, dropout)
    "b0": (0, 224, 0.2),
    "b1": (0.5, 240, 0.2),
    "b2": (1, 260, 0.3),
    "b3": (2, 300, 0.3),
    "b4": (3, 380, 0.4),
    "b5": (4, 456, 0.4),
    "b6": (5, 528, 0.5),
    "b7": (6, 600, 0.5),
}

In [11]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, 
                 stride, padding, groups=1):
        super(ConvBlock, self).__init__()
        self.cnnblock = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size,
                                  stride, padding, groups=groups),
                        nn.BatchNorm2d(out_channels),
                        nn.SiLU())

    def forward(self, x):
        return self.cnnblock(x)

In [12]:
class MBBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size,
        stride, padding, ratio, reduction=2,
    ):
        super(MBBlock, self).__init__()
        # self.use_residual = in_channels == out_channels and stride == 1
        hidden_dim = in_channels * ratio
        self.expand = in_channels != hidden_dim

        # This is for squeeze and excitation block
        reduced_dim = int(in_channels / reduction)

        if self.expand:
            self.expand_conv = ConvBlock(in_channels, hidden_dim,
                kernel_size=3,stride=1,padding=1)
#         print(hidden_dim,out_channels)
        self.conv = nn.Sequential(
                ConvBlock(hidden_dim,hidden_dim,kernel_size,
                  stride,padding,groups=hidden_dim),
                SqueezeExcitation(hidden_dim, reduced_dim),
                nn.Conv2d(hidden_dim, out_channels, 1),
                nn.BatchNorm2d(out_channels),
            )

    def forward(self, inputs):
        if self.expand:
            x = self.expand_conv(inputs)
        else:
            x = inputs
        return self.conv(x)

In [13]:
class SqueezeExcitation(nn.Module):
    def __init__(self, in_channels, reduced_dim):
        super(SqueezeExcitation, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),  # C x H x W -> C x 1 x 1
            nn.Conv2d(in_channels, reduced_dim, 1),
            nn.SiLU(),
            nn.Conv2d(reduced_dim, in_channels, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return x * self.se(x)

In [29]:
class EfficientNet(nn.Module):
    def __init__(self, model_name, output):
        super(EfficientNet, self).__init__()
#         phi, resolution, dropout = scale_values[model_name]
        self.depth_factor, self.width_factor = 1,1
        self.last_channels = 1280
        self.avgpool= nn.AdaptiveAvgPool2d(1)
        self.feature_extractor()
        self.flatten = nn.Flatten()
        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.last_channels, output),
        )

    def feature_extractor(self):
        channels = int(32 * self.width_factor)
        features = [ConvBlock(4, channels, 3, stride=2, padding=1,groups=4)]
        in_channels = channels

        for k, c_o, repeat, s, n in basic_mb_params:
            # For numeric stability, we multiply and divide by 4
            out_channels = int(4 * np.ceil(int(c_o * self.width_factor) / 4))
            num_layers = np.ceil(repeat * self.depth_factor)

            for layer in range(int(num_layers)):
                if layer == 0:
                    stride = s
                else:
                    stride = 1
                features.append(
                        MBBlock(in_channels,out_channels,ratio=k,
                        stride=stride,kernel_size=n,padding=n// 2)
                    )
                in_channels = out_channels

        features.append(
            ConvBlock(in_channels, self.last_channels, 
            kernel_size=1, stride=1, padding=0)
        )
        self.extractor = nn.Sequential(*features)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
#         print(x.shape)
        x = self.avgpool(self.extractor(x))
        return self.softmax( self.classifier(self.flatten(x)) )

model_name = 'b0'
output_class = 6 


In [30]:
def validatn_loss(data_loader, model):
        
    model.to(device)
    model.eval()    
    predictions = []
    true = []
#     loss_lst = []
    for batch in data_loader:
        with torch.no_grad():
            x, y = batch
            x = x.to(device)
            # inputs = {key:val.reshape(val.shape[0], -1).to(config.device) for key,val in batch.items()}
            outputs = model(x)
        predictions.extend(outputs.detach())
        true.extend(y)
#         loss_lst.extend(criterion(torch.log(outputs.detach()), y.to(device)))
        

    predictions = torch.vstack(predictions)
    true = torch.vstack(true)
#     loss_lst = torch.vstack(loss_lst)
#     print(loss_lst.mean())
    loss = criterion(torch.log(predictions), true.to(device))
#     print(loss)
    return loss.item()

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [32]:
if not os.path.exists('cnn2d_model'):
        os.makedirs('cnn2d_model')

In [ ]:
our_model = EfficientNet(model_name, output_class)    
our_model.to(device)

if IS_TRAINING:
    gkf = GroupKFold(n_splits=5)
    for i, (train_index, valid_index) in enumerate(gkf.split(train, train.target, train.patient_id)):
        dataset_train = CustomDataset(dataframe=train.iloc[train_index],specs = spectrograms)
        dataset_test = CustomDataset(dataframe=train.iloc[valid_index],specs = spectrograms)
        train_dataloader = DataLoader(dataset_train, batch_size=16, shuffle=True,drop_last=True)
        val_loader = DataLoader(dataset_test, batch_size=16,shuffle=False)

        min_val_loss = 99
        epochs = 6
#         our_model.pretrained_model.load_state_dict(torch.load(file_name))
#         our_model = EfficientNet(model_name, output_class)    
#         our_model.to(device)
        optimizer = optim.Adam(our_model.parameters(), lr=0.001)
        criterion = nn.KLDivLoss(reduction="batchmean").cuda()
#         our_model.to(torch.float32)
        for epoch in range(epochs):
            our_model.train()
            pbar = tqdm(train_dataloader)
            running_loss=0
            cnt = 0
            for batch in pbar:
                inp, label = batch
#                 print(inp.shape)
                pred = our_model(inp.to(device))
                loss = criterion(torch.log(pred), label.to(device))
                # Backward pass and optimization
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                running_loss += loss*inp.size(0)/len(train_dataloader.dataset)

                pbar.set_description(f"Batch loss : | Runn: {loss.item():.2f} | {running_loss.item():.2f}")
                cnt+=1
                if cnt == len(train_dataloader)//2:
                    val_loss = validatn_loss(val_loader,our_model)
                    if min_val_loss > val_loss :
                        min_val_loss = val_loss
                        torch.save(our_model.state_dict(), f'cnn2d_model/model_best_fold_{i}.pt')
                        print('i,epoch_half,val loss : ',i,epoch,val_loss)
                    our_model.train()
                    
            val_loss = validatn_loss(val_loader,our_model)
            if min_val_loss > val_loss :
                min_val_loss = val_loss
                torch.save(our_model.state_dict(), f'cnn2d_model/model_best_fold_{i}.pt')
                print('i,epoch,val loss : ',i,epoch,val_loss)
                

Batch loss : | Runn: 1.15 | 0.24:  18%|█▊        | 154/854 [00:25<01:53,  6.17it/s]

# Submit to Kaggle LB


In [ ]:
# del train; gc.collect()
test = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/test.csv')
print('Test shape:',test.shape)
test.head()

In [ ]:
# READ ALL SPECTROGRAMS
PATH2 = '/kaggle/input/hms-harmful-brain-activity-classification/test_spectrograms/'
files2 = os.listdir(PATH2)
print(f'There are {len(files2)} test spectrogram parquets')
    
spectrograms2 = {}
for i,f in enumerate(files2):
    if i%100==0: print(i,', ',end='')
    tmp = pd.read_parquet(f'{PATH2}{f}')
    name = int(f.split('.')[0])
    spectrograms2[name] = tmp.iloc[:,1:].values
    
# RENAME FOR DATALOADER
test = test.rename({'spectrogram_id':'spec_id'},axis=1)

In [ ]:
dataset_test = CustomDataset(dataframe=test,specs = spectrograms2,mode='Test')

In [ ]:
test_loader = DataLoader(dataset_test, batch_size=8,shuffle=False)
# model = EffNET_HMS().double()

eff_model = EfficientNet.from_name(model_name)
eff_model.load_state_dict(torch.load(file_name))
eff_model._fc = nn.Linear(eff_model._fc.in_features, 6,bias=True)
#         our_model.pretrained_model.load_state_dict(torch.load(file_name))
our_model = EffNET_HMS(eff_model)

preds_all_fold = []
for i in range(5):
    our_model.load_state_dict(torch.load('/kaggle/input/effnet-models/model_best_fold_'+str(i)+'.pt'))
    our_model.to(device)
    our_model.eval()
    
    preds = []
    for batch in test_loader:
        pred = our_model(batch.to(device))
        preds.append(pred.detach().cpu().numpy())
    preds = np.vstack(preds)
    preds_all_fold.append(preds)

prediction_all_fold = np.mean(preds_all_fold,axis=0)

In [ ]:
# CREATE SUBMISSION.CSV
from IPython.display import display

sub = pd.DataFrame({'eeg_id':test.eeg_id.values})
sub[TARGETS] = prediction_all_fold
sub.to_csv('submission.csv',index=False)
print('Submission shape',sub.shape)
display( sub.head() )

# SANITY CHECK TO CONFIRM PREDICTIONS SUM TO ONE
print('Sub row 0 sums to:',sub.iloc[0,-6:].sum())